In [144]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline 

import numpy as np
import time
from torchvision import datasets, transforms

In [145]:
batch_size = 32
learning_rate = 0.0001

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


## Load In Data

In [146]:
transform = transforms.Compose([transforms.Resize(224),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()
                               ])

In [147]:
train_dataset = datasets.ImageFolder("data/train", transform=transform) 
train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
train_iter = iter(train_dl)

val_dataset = datasets.ImageFolder("data/valid", transform=transform) 
val_dl = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)
val_iter = iter(val_dl)

test_seen_dataset = datasets.ImageFolder("data/test-seen", transform=transform) 
test_seen_dl = torch.utils.data.DataLoader(test_seen_dataset, batch_size=32, shuffle=True)
test_seen_iter = iter(test_seen_dl)

test_unseen_dataset = datasets.ImageFolder("data/test-unseen", transform=transform) 
test_unseen_dl = torch.utils.data.DataLoader(test_unseen_dataset, batch_size=32, shuffle=True)
test_unseen_iter = iter(test_unseen_dl)

## Helper Functions 

In [148]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
device = get_default_device()

In [161]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [149]:
class Accumulator:
  def __init__(self, n):
    self.data = [0.0] * n

  def add(self, *args):
    self.data = [a + float(b) for a, b in zip(self.data, args)]

  def __getitem__(self, i):
    return self.data[i]

In [150]:
def accuracy(y_hat, y):
    """Compute the number of correct predictions."""
    #y_hat is batch_size by n_outputs and y is n_outputs
    # y contains the index of the ground truth label

    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1: #The output is a vector -- not just a scalar
        y_hat = y_hat.argmax(axis=1)
        
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

## Network Definition

In [151]:
#create network

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 6, 5)
        self.fc1 = nn.Linear(6 * 110 * 110, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 10)


    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Training Loop Definition

In [168]:
def train_one_epoch(net, train_iter, loss, updater):
  net.train()
  net.to(device)

  metric = Accumulator(3)

  # for each batch in the training set
  for inputs, labels in train_iter:
    #inputs, labels = images.to(device), labels.to(device)
    inputs = to_device(inputs, device)
    labels = to_device(labels, device)
      
    if(inputs.shape[0] > labels.shape[0]):
        inputs = inputs[:labels.shape[0],:,:,:]
      
    net_output = net(inputs)

    l = loss(net_output, labels)
    acc = accuracy(net_output, labels)

    updater.zero_grad()

    l.backward()

    updater.step()

    #adds loss for whole batch and batch size
    metric.add(float(l) * len(labels), float(acc), labels.numel())

  return metric[0] / metric[2], metric[1] / metric[2]

In [169]:
def train(net, epochs, dl, loss, trainer):
    for epoch in range(epochs):

        start_time = time.time()
        history = train_one_epoch(net, iter(dl), loss, trainer)
        epoch_time = time.time() - start_time
        
        print("Epoch: {} -- Train Loss: {:.3f} Train Accuracy: {:.3f} Time: {:.3f} Sec".format(epoch, history[0], history[1], epoch_time))

## Interaction and Training

In [170]:
net = Net()
loss = nn.CrossEntropyLoss()
trainer = torch.optim.SGD(net.parameters(), lr=0.01)

train(net, 15, train_dl, loss, trainer)

Epoch: 0 -- Train Loss: 0.998 Train Accuracy: 0.485 Time: 4.032 Sec
Epoch: 1 -- Train Loss: 0.717 Train Accuracy: 0.569 Time: 3.823 Sec
Epoch: 2 -- Train Loss: 0.778 Train Accuracy: 0.610 Time: 3.965 Sec
Epoch: 3 -- Train Loss: 0.756 Train Accuracy: 0.568 Time: 3.978 Sec
Epoch: 4 -- Train Loss: 0.720 Train Accuracy: 0.552 Time: 4.201 Sec
Epoch: 5 -- Train Loss: 0.654 Train Accuracy: 0.631 Time: 4.141 Sec
Epoch: 6 -- Train Loss: 0.665 Train Accuracy: 0.658 Time: 4.119 Sec
Epoch: 7 -- Train Loss: 0.588 Train Accuracy: 0.718 Time: 4.003 Sec
Epoch: 8 -- Train Loss: 0.583 Train Accuracy: 0.751 Time: 3.911 Sec
Epoch: 9 -- Train Loss: 0.618 Train Accuracy: 0.687 Time: 4.052 Sec
Epoch: 10 -- Train Loss: 0.517 Train Accuracy: 0.763 Time: 3.990 Sec
Epoch: 11 -- Train Loss: 0.549 Train Accuracy: 0.728 Time: 4.003 Sec
Epoch: 12 -- Train Loss: 0.551 Train Accuracy: 0.745 Time: 4.061 Sec
Epoch: 13 -- Train Loss: 0.512 Train Accuracy: 0.753 Time: 3.941 Sec
Epoch: 14 -- Train Loss: 0.554 Train Accurac